# Heat equation example

## Analytic problem formulation

We consider the heat equation on the segment $[0, 1]$, with dissipation on both sides, heating (input) $u$ on the left, and measurement (output) $\tilde{y}$ on the right:
$$
\begin{align*}
    \partial_t T(z, t) & = \partial_{zz} T(z, t), & 0 < z < 1,\ t > 0, \\
    \partial_z T(0, t) & = T(0, t) - u(t), & t > 0, \\
    \partial_z T(1, t) & = -T(1, t), & t > 0, \\
    \tilde{y}(t) & = T(1, t), & t > 0.
\end{align*}
$$

## Semidiscretized formulation

By central finite differences on the equidistant mesh $0 = z_1 < z_2 < \ldots < z_n = 1$, we obtain the semidiscretized formulation:
$$
\begin{align*}
    \dot{x}_i(t) & = \frac{x_{i - 1}(t) - 2 x_i(t) + x_{i + 1}(t)}{h^2}, & i = 1, 2, 3, \ldots, n - 1, n, \\
    \frac{x_2(t) - x_0(t)}{2 h} & = x_1(t) - u(t), \\
    \frac{x_{n + 1}(t) - x_{n - 1}(t)}{2 h} & = -x_n(t), \\
    y(t) & = x_n(t),
\end{align*}
$$
where $h = \frac{1}{n - 1}$, $x_i(t) \approx T(z_i, t)$, and $y(t) \approx \tilde{y}(t)$.

Separating cases $i = 1$ and $i = n$ in the first equation, we find:
$$
\begin{alignat*}{3}
    \dot{x}_1(t) & = && -2 n (n - 1) x_1(t) && + 2 (n - 1)^2 x_2(t) + 2 (n - 1) u(t), \\
    \dot{x}_i(t) & = (n - 1)^2 x_{i - 1}(t) && - 2 (n - 1)^2 x_i(t) && + (n - 1)^2 x_{i + 1}(t),
    && i = 2, 3, \ldots, n - 1, \\
    \dot{x}_n(t) & = 2 (n - 1)^2 x_{n - 1}(t) && - 2 n (n - 1)  x_n(t), \\
    y(t) & = x_n(t).
\end{alignat*}
$$

## Import modules

In [ ]:
from __future__ import absolute_import, division, print_function

import numpy as np
import scipy.sparse as sps
import matplotlib.pyplot as plt
%matplotlib inline

from pymor.discretizations.iosys import LTISystem
from pymor.reductors.bt import bt
from pymor.reductors.lti import irka, tsia

import logging
logging.getLogger('pymor.algorithms.gram_schmidt.gram_schmidt').setLevel(logging.ERROR)

## Assemble $A$, $B$,  and $C$

In [ ]:
n = 100  # dimension of the system

A = sps.diags([n * [-2 * (n - 1) ** 2],
               (n - 1) * [(n - 1) ** 2],
               (n - 1) * [(n - 1) ** 2]],
              [0, -1, 1],
              format='csc')
A[0, 0] = -2 * n * (n - 1)
A[0, 1] *= 2
A[-1, -1] = -2 * n * (n - 1)
A[-1, -2] *= 2

B = np.zeros((n, 1))
B[0, 0] = 2 * (n - 1)

C = np.zeros((1, n))
C[0, n - 1] = 1

## LTI system

In [ ]:
lti = LTISystem.from_matrices(A, B, C)

print('n = {}'.format(lti.n))
print('m = {}'.format(lti.m))
print('p = {}'.format(lti.p))

In [ ]:
poles = lti.poles()
fig, ax = plt.subplots()
ax.plot(poles.real, poles.imag, '.')
ax.set_title('System poles')
plt.show()

In [ ]:
w = np.logspace(-2, 3, 100)
fig, ax = LTISystem.mag_plot(lti, w=w)
ax.set_title('Bode plot of the full model')
plt.show()

In [ ]:
sv = lti.sv_U_V('lyap')[0]
fig, ax = plt.subplots()
ax.semilogy(range(1, len(sv) + 1), sv, '.-')
ax.set_title('Hankel singular values')
plt.show()

In [ ]:
print('H_2-norm of the full model:   {}'.format(lti.norm()))
print('H_inf-norm of the full model: {}'.format(lti.norm('Hinf')))

## Balanced Truncation (BT)

In [ ]:
r = 5
rom_bt, _ = bt(lti, r, tol=1e-5)

In [ ]:
print('H_2-norm of the BT ROM:       {}'.format(rom_bt.norm()))
print('H_inf-norm of the BT ROM:     {}'.format(rom_bt.norm('Hinf')))
err_bt = lti - rom_bt
print('H_2-error for the BT ROM:     {}'.format(err_bt.norm()))
print('H_inf-error for the BT ROM:   {}'.format(err_bt.norm('Hinf')))

In [ ]:
fig, ax = LTISystem.mag_plot((lti, rom_bt), w=w)
ax.set_title('Bode plot of the full and BT reduced model')
plt.show()

In [ ]:
fig, ax = LTISystem.mag_plot(err_bt, w=w)
ax.set_title('Bode plot of the BT error system')
plt.show()

## Iterative Rational Krylov Algorithm (IRKA)

In [ ]:
sigma = np.logspace(-1, 3, r)
tol = 1e-4
maxit = 100
rom_irka, _, reduction_data_irka = irka(lti, r, sigma, tol=tol, maxit=maxit, verbose=True, compute_errors=True)

In [ ]:
fig, ax = plt.subplots()
ax.semilogy(reduction_data_irka['dist'], '.-')
ax.set_title('Distances between shifts in IRKA iterations')
plt.show()

In [ ]:
print('H_2-norm of the IRKA ROM:     {}'.format(rom_irka.norm()))
print('H_inf-norm of the IRKA ROM:   {}'.format(rom_irka.norm('Hinf')))
err_irka = lti - rom_irka
print('H_2-error for the IRKA ROM:   {}'.format(err_irka.norm()))
print('H_inf-error for the IRKA ROM: {}'.format(err_irka.norm('Hinf')))

In [ ]:
fig, ax = LTISystem.mag_plot((lti, rom_irka), w=w)
ax.set_title('Bode plot of the full and IRKA reduced model')
plt.show()

In [ ]:
fig, ax = LTISystem.mag_plot(err_irka, w=w)
ax.set_title('Bode plot of the IRKA error system')
plt.show()

## Two-Sided Iteration Algorithm (TSIA)

In [ ]:
Ar = np.diag(-np.logspace(-1, 3, r))
Br = np.ones((r, 1))
Cr = np.ones((1, r))
Er = np.eye(r)

In [ ]:
rom0 = LTISystem.from_matrices(Ar, Br, Cr, E=Er)

In [ ]:
rom_tsia, _, rd_tsia = tsia(lti, rom0, verbose=True, compute_errors=True)